<h1>Bases des télécommunications 1</h1>

**__Introduction__**

Le but de ce laboratoire est de mettre en pratique les concepts vus en classe de Bases des
Télécommunications. On travaillera ainsi sur la détection d'erreur sur un fichier texte.

- Le rendu du **jupyter notebook** doit se faire sur __cyberlearn__ avec comme nom "prenom_nom_groupe" 
- Le fichier texte utilisé ici doit être encodé en UTF-8 (sans BOM), vous pouvez le vérifier avec Notepad++ par exemple.
- Exemple de code python 3 pour ouvrir en lecture un fichier texte en UTF-8 : __*open(INPUT, "r", encoding="UTF-8")*__
<h2>I.Contrôle d'erreur (CRC)</h2>

a) Réaliser ci-dessous une **fonction** qui génère le bit de parité d'un charactère

<table style=" margin-left: auto;margin-right: auto;">
    <tr>
        <td>2<sup>7</sup></td>
        <td>2<sup>6</sup></td>
        <td>2<sup>5</sup></td>
        <td>2<sup>4</sup></td>
        <td>2<sup>3</sup></td>
        <td>2<sup>2</sup></td>
        <td>2<sup>1</sup></td>
        <td>2<sup>0</sup></td>
    </tr>
    <tr>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
    </tr>        
    <tr>
        <td colspan="8" style=" text-align: right;">↑ <br/> bit de parité</td>
    </tr>        
</table>

Voici des exemples qui vont permettre de comprendre comment fonctionne la création du bit de parité. **CONSEIL: Un charactère ascii est codé sur combien de bit?**
<table style=" margin-left: auto;margin-right: auto;">
    <tr>
    <td colspan="8" style="text-align:center" >charactère '*'</td>
    </tr>
    <tr>
        <td>2<sup>7</sup></td>
        <td>2<sup>6</sup></td>
        <td>2<sup>5</sup></td>
        <td>2<sup>4</sup></td>
        <td>2<sup>3</sup></td>
        <td>2<sup>2</sup></td>
        <td>2<sup>1</sup></td>
        <td>2<sup>0</sup></td>
    </tr>
    <tr>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
    </tr>        
    <tr>
        <td colspan="8" style=" text-align: right;">↑ <br/> bit de parité</td>
    </tr>        
</table>

</br>
<table style=" margin-left: auto;margin-right: auto;">
    <tr>
    <td colspan="8" style="text-align:center" >charactère '('</td>
    </tr>
    <tr>
        <td>2<sup>7</sup></td>
        <td>2<sup>6</sup></td>
        <td>2<sup>5</sup></td>
        <td>2<sup>4</sup></td>
        <td>2<sup>3</sup></td>
        <td>2<sup>2</sup></td>
        <td>2<sup>1</sup></td>
        <td>2<sup>0</sup></td>
    </tr>
    <tr>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
    </tr>        
    <tr>
        <td colspan="8" style=" text-align: right;">↑ <br/> bit de parité</td>
    </tr>        
</table>

In [2]:
#la variable char est de type int
def generate_parity_bit(char):
  parity_bit = 0
  bits = format(char, 'b')
  bits = bits[:-1]
  for bit in bits:
    parity_bit = parity_bit ^ int(bit)
  return parity_bit

b) Réaliser ci-dessous un programme qui Affiche le nombre total d'erreur dans le fichier text (**input_with_error.bin**) à l'aide du bit de parité.

In [3]:
error_count = 0
file_with_error = open("input_with_error.bin", "rb")
for line in file_with_error:
    for character in line:
      bits = format(character, 'b')
      file_parity_bit = int(bits[len(bits)-1:])
      if file_parity_bit != generate_parity_bit(character):
        error_count += 1

print("Nombre d'erreurs: ", error_count)

Nombre d'erreurs:  42123


c) Réaliser ci-dessous une fonction permettant:
 
 1. de calculer le bit de parité du charactère
 2. d'altérer le charactère _**dans toutes les positions possibles selon la table ASCII**_ (Conseil: utilisé la fonction **[randrange()](https://docs.python.org/3/library/random.html#random.randrange)** 
 3. ajouter le bit de parité au charctère altéré.

In [4]:
def edit_char(char):
  new_char = rndm.randrange(128)
  new_char = format(new_char, 'b')
  new_char += str(get_parity_bit(char))
  return new_char

d) Réaliser ci-dessous un programme permettant d'ajouter le bit de parité selon le système d'état suivant au fichier **"input.txt"** selon le schéma suivant et sauvegarder le résultat **"output.bin"**:
<img src="scheme_crc.jpg"/>


La **seed** utilisé pour créer le fichier **input_with_error.bin** est le numéro **43**.

In [5]:
import random as rndm

file_input = open("input.txt", "rb")
file_output = open("output.bin", "wb")

def get_parity_bit(char):
  parity_bit = 0
  bits = format(char, 'b')
  for bit in bits:
    parity_bit ^= int(bit)
  return parity_bit

def add_parity_bit(char):
  parity_bit = get_parity_bit(char)
  char = format(char, 'b')
  char += str(parity_bit)
  return int(char, 2)

CreateError = False
output_data = bytearray(file_input.read())
index = 0
rndm.seed(43)
for character in output_data:
    if CreateError:
      character = int(edit_char(character), 2)
    else:
      character = add_parity_bit(character)
    output_data[index] = character
    index += 1
    if rndm.random() > 1 - 10E-6:
      CreateError = not CreateError

file_output.write(output_data)

file_with_error.close()
file_input.close()
file_output.close()

**ATTENTION**,lorsque vous utilisez le random avec la seed, pensez à reéxecuter LE CODE DE LA SEED car sinon vous aurez des valeurs différentes que le fichier encodé par nos soins. 

<h2> II. Bonus </h2>
vérifier de deux manières différentes que votre fichier générer est identique que le fichier **input_with_error.txt**
a) vérification 1

In [ ]:
# Comparaison des caractères

file_origin = open("input_with_error.bin", "rb")
file_new = open("output.bin", "rb")
are_equal = True

while True:
    char_og = file_origin.read()
    char_n = file_new.read()
    if char_og != char_n:
        are_equal = False
    if char_og == '':
        break
        
    
print(f"les fichiers sont :")
if are_equals:
    print("identiques")
else:
    print("différents")

b)vérification 2

In [ ]:
# Comparaison du bit de parité

are_equal = true


while True:
    char_og = file_origin.read()
    char_n = file_new.read()
    if get_parity_bit(char_og) != get_parity_bit(char_n):
        are_equal = False
    if char_og == '':
        break

print(f"les fichiers sont :")
if are_equals:
    print("identiques")
else:
    print("différents")

file_origin.close()
file_new.close()

c) actuellement le code génère plus d'erreurs, comment peut-on diminuer ces erreurs 

In [ ]:
En modifiant la valeur de "1 - 10E-6"